In [6]:
from pathlib import Path

import pandas as pd


In [7]:
import os

In [8]:
data = pd.read_csv(Path(os.path.expandvars("$MOVIES_DATA/scenes/scenes.csv")))

In [9]:
data

,Unnamed: 0,text,start,end,n_lines,movie_id,text_len
0,0,Lift your leg up. Is this seat taken? What's g...,67.000,274.028,31,tt2124074,895
1,1,Hey. Hey. I gotta get down to the lobby. Some...,343.659,521.091,30,tt2124074,812
2,2,What? Tell me something about yourself. Like...,542.279,592.058,17,tt2124074,565
3,3,Hold on. I'm thinking. Only one? Don't push it...,609.240,713.371,19,tt2124074,638
4,4,"See ya. I, uh I read your book. Oh, yeah? It...",746.034,799.751,18,tt2124074,596
...,...,...,...,...,...,...,...
153723,153723,"Hey, Ralph! Hello, Florence. Gee, am I glad ...",3950.660,3978.860,11,tt0024368,518
153724,153724,All right. I'll talk. I'll tell you what I kno...,4006.820,4067.220,21,tt0024368,1001
153725,153725,Your face... was of wax. You fiend. My Marie...,4085.660,4132.340,12,tt0024368,373
153726,153726,"Charlotte! Don't be afraid, my dear. In a few ...",4145.940,4274.620,7,tt0024368,301


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length = 512)

In [4]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [5]:
from tqdm import tqdm
tqdm.pandas()

In [10]:
inds, texts = zip(*sorted(enumerate(data["text"]), key=lambda x: len(str(x[1]))))

In [11]:
import math

In [ ]:
batch_size = 128
summaries = []
# for i in tqdm(range(math.ceil(len(texts)/batch_size))):
for i in tqdm(range(422,math.ceil(len(texts)/batch_size))):
    tt = ["summarize: " + str(x) for x in texts[i*batch_size : min(len(texts), (i+1)*batch_size)]]
    input_ids =tokenizer(tt, return_tensors="pt", padding=True, max_length = 512, truncation=True).input_ids
    outputs = model.generate(input_ids, max_length=40, min_length=10, do_sample=False)
    summaries.extend([tokenizer.decode(o, skip_special_tokens=True) for o in outputs])


  0%|                                                   | 0/779 [00:00<?, ?it/s]

In [3]:
help(tokenizer)

Help on T5Tokenizer in module transformers.models.t5.tokenization_t5 object:

class T5Tokenizer(transformers.tokenization_utils.PreTrainedTokenizer)
 |  T5Tokenizer(vocab_file, eos_token='</s>', unk_token='<unk>', pad_token='<pad>', extra_ids=100, additional_special_tokens=None, sp_model_kwargs: Optional[Dict[str, Any]] = None, **kwargs) -> None
 |  
 |  Construct a T5 tokenizer. Based on [SentencePiece](https://github.com/google/sentencepiece).
 |  
 |  This tokenizer inherits from [`PreTrainedTokenizer`] which contains most of the main methods. Users should refer to
 |  this superclass for more information regarding those methods.
 |  
 |  Args:
 |      vocab_file (`str`):
 |          [SentencePiece](https://github.com/google/sentencepiece) file (generally has a *.spm* extension) that
 |          contains the vocabulary necessary to instantiate a tokenizer.
 |      eos_token (`str`, *optional*, defaults to `"</s>"`):
 |          The end of sequence token.
 |  
 |          <Tip>
 |  


In [ ]:
print(len(summaries))